# Lockdown baking - part 1

A project for scraping and analysing data from reddit (r/Sourdough) to explore baking trends during 2020.

Part 1: Webscraping

- setting up reddit API account
- creating functions for extracting data using [praw]( https://praw.readthedocs.io/en/latest/code_overview/models/submission.html) and [pushshift.io](https://pushshift.io/api-parameters/) 
- storing result in a csv file 

# Setup

In [7]:
import pandas as pd
import numpy as np
import os #to change file paths
import configparser #to read config file
import praw #to access reddit API
import pickle #to store objects
import math
import json
import requests
import itertools
import time
from datetime import datetime, timedelta

In [5]:
## go to root folder
os.chdir("..")

## Setting up reddit API connection

In [47]:
# retrieve details from config file
def get_config_values(config_file, section):
    config = configparser.ConfigParser()
    config.read(config_file)

    return {
        "username": config.get(section, 'username'),
        "password": config.get(section, 'password'),
        "user_agent": config.get(section, 'user_agent'),
        "client_id": config.get(section, 'client_id'),
        "client_secret": config.get(section, 'client_secret'),
    }

details = get_config_values("reddit-config.cfg", "reddit-config")

In [48]:
# setup praw Reddit connection
reddit = praw.Reddit(client_id = details["client_id"], 
                     client_secret = details["client_secret"], 
                     user_agent = details["user_agent"], 
                     username = details["username"], 
                     password = details["password"]) 
  
# to verify whether the instance is authorised instance or not 
print(reddit.read_only)

False


In [46]:
details["client_id"]

'vNW8cTO2bw2jJA'

In [29]:
config = configparser.ConfigParser()
config.read("reddit-config.cfg")
username = config.get('reddit-config', 'username')
password = config.get('reddit-config', 'password')

'Winter05'

In [33]:
reddit = praw.Reddit(client_id ='vNW8cTO2bw2jJA', 
                     client_secret ='2abo8mFQWCHczgBIMha34yqcjdE0Jw', 
                     user_agent ='my user agent', 
                     username ='liloho', 
                     password ='Winter05') 
  
# to verify whether the instance is authorised instance or not 
print(reddit.read_only) 

False


In [ ]:
from configparser import ConfigParser
cfg = ConfigParser()
cfg.read('config.ini')
print('Sections in the file:', cfg.sections())
print('hms: form value: ', cfg.get('hms','form'))
print('hms: splashScreen value: ', cfg.getboolean('hms','splashScreen'))
print('webutil: WebUtilDispatchMonitorInterval value: ', cfg.getint('webutil','WebUtilDispatchMonitorInterval'))

Output:

Sections in the file: ['sepwin', 'hms', 'debug', 'webutil']
hms: form value:  hospital.fmx
hms: splashScreen value:  False
webutil: WebUtilDispatchMonitorInterval value:  5

In [ ]:
#Importing the modules
import os
import ConfigParser
import time

# On Debian, /etc/mysql/debian.cnf contains 'root' a like login and password.
config = ConfigParser.ConfigParser()
config.read("/etc/mysql/debian.cnf")
username = config.get('client', 'user')
password = config.get('client', 'password')
hostname = config.get('client', 'host')



filestamp = time.strftime('%Y-%m-%d')